# Practica 2: Predicción de Series Temporales

# Autores: 

- Alejandro López Cuéllar
- Miguel Ángel Vera Frutos
- Alejandro Belda Fernández


# Importación de librerías

In [23]:
import pandas as pd
import os
import locale

In [24]:
locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")  # Ajusta "es_ES.UTF-8" según tu sistema operativo.
locale.setlocale(locale.LC_TIME, "Spanish_Spain.1252")

'Spanish_Spain.1252'

# Definición de la ruta

In [25]:
ruta = "Data/Aljorrahorarias2017.xls"
ruta2 = "Data/Aljorrahorarias2018.xls"
ruta3 = "Data/Aljorrahorarias2019.xls"
ruta4 = "Data/Aljorrahorarias2020.xls"
ruta5 = "Data/Aljorrahorarias2021.xls"
ruta6 = "Data/Aljorrahorarias2022.xls"


In [26]:
# Leer el archivo completo
df = pd.read_excel(ruta, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
df["Fecha"] = df["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df.set_index("Fecha", inplace=True)


# Leer el archivo completo
df2 =  pd.read_excel(ruta2, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
df2["Fecha"] = df2["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df2.set_index("Fecha", inplace=True)

# Leer el archivo completo
df3 =  pd.read_excel(ruta3, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
df3["Fecha"] = df3["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df3.set_index("Fecha", inplace=True)

# Leer el archivo completo
df4 =  pd.read_excel(ruta4, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
df4["Fecha"] = df4["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df4.set_index("Fecha", inplace=True)

# Leer el archivo completo
df5 =  pd.read_excel(ruta5, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
# Establecer los minutos a 00, y redondear la fecha a la hora
df5["Fecha"] = df5["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df5.set_index("Fecha", inplace=True)

# Leer el archivo completo
df6 =  pd.read_excel(ruta6, engine='xlrd', skiprows=1, parse_dates=["Fecha"], date_format="%d %b %Y %H:%M")
# Establecer los minutos a 00, y redondear la fecha a la hora
df6["Fecha"] = df6["Fecha"].dt.floor('H')

# Establecer "Fecha" como el índice
df6.set_index("Fecha", inplace=True)




*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_27476\1045028487.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df["Fecha"] = df["Fecha"].dt.floor('H')


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_27476\1045028487.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2["Fecha"] = df2["Fecha"].dt.floor('H')


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


...

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27476\1045028487.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df4["Fecha"] = df4["Fecha"].dt.floor('H')


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_27476\1045028487.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df5["Fecha"] = df5["Fecha"].dt.floor('H')


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


C:\Users\Usuario\AppData\Local\Temp\ipykernel_27476\1045028487.py:41: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df6["Fecha"] = df6["Fecha"].dt.floor('H')


In [27]:
data= pd.concat([df,df2,df3,df4,df5,df6])

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52584 entries, 2017-01-01 00:00:00 to 2022-12-31 23:00:00
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NO (µg/m³N)    47992 non-null  float64
 1   NO2 (µg/m³N)   47992 non-null  float64
 2   SO2 (µg/m³N)   46390 non-null  float64
 3   O3 (µg/m³N)    46626 non-null  float64
 4   TMP (ºC)       51251 non-null  float64
 5   HR (% H.R.)    51251 non-null  float64
 6   NOX (µg/m³N)   47992 non-null  float64
 7   DD (grados)    51251 non-null  float64
 8   PRB (mb)       51251 non-null  float64
 9   RS (W/m³)      51251 non-null  float64
 10  VV (m/s)       51252 non-null  float64
 11  C6H6 (µg/m³N)  14069 non-null  float64
 12  C7H8 (µg/m³N)  14069 non-null  float64
 13  XIL (µg/m³N)   14069 non-null  float64
 14  PM10 (µg/m³N)  49744 non-null  float64
 15  Ruido (dBA)    0 non-null      float64
dtypes: float64(16)
memory usage: 6.8 MB


In [29]:
data.asfreq('1h')
# data.asfreq("D")

,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
Fecha,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,5.0,30.0,15.0,43.0,10.0,85.0,37.0,299.0,1031.0,2.0,1.0,NaN,NaN,NaN,26.00,NaN
2017-01-01 01:00:00,4.0,27.0,12.0,41.0,9.0,84.0,34.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,25.00,NaN
2017-01-01 02:00:00,5.0,25.0,14.0,39.0,9.0,86.0,33.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,21.00,NaN
2017-01-01 03:00:00,3.0,16.0,15.0,43.0,8.0,85.0,21.0,301.0,1029.0,3.0,1.0,NaN,NaN,NaN,18.00,NaN
2017-01-01 04:00:00,4.0,15.0,14.0,46.0,8.0,84.0,22.0,302.0,1029.0,2.0,1.0,NaN,NaN,NaN,16.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,2.5,10.0,4.6,57.6,16.0,77.0,13.9,185.0,1024.0,2.0,1.2,NaN,NaN,NaN,31.75,NaN
2022-12-31 20:00:00,4.6,17.5,5.0,28.2,14.0,81.0,24.6,235.0,1025.0,2.0,1.0,NaN,NaN,NaN,53.27,NaN
2022-12-31 21:00:00,3.9,12.9,4.9,27.4,13.0,88.0,18.8,280.0,1025.0,2.0,1.0,NaN,NaN,NaN,55.28,NaN


In [30]:
data.head()

,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N),Ruido (dBA)
Fecha,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,5.0,30.0,15.0,43.0,10.0,85.0,37.0,299.0,1031.0,2.0,1.0,NaN,NaN,NaN,26.0,NaN
2017-01-01 01:00:00,4.0,27.0,12.0,41.0,9.0,84.0,34.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,25.0,NaN
2017-01-01 02:00:00,5.0,25.0,14.0,39.0,9.0,86.0,33.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,21.0,NaN
2017-01-01 03:00:00,3.0,16.0,15.0,43.0,8.0,85.0,21.0,301.0,1029.0,3.0,1.0,NaN,NaN,NaN,18.0,NaN
2017-01-01 04:00:00,4.0,15.0,14.0,46.0,8.0,84.0,22.0,302.0,1029.0,2.0,1.0,NaN,NaN,NaN,16.0,NaN


In [31]:
fechas_esperadas =  pd.date_range(start=data.index.min(), end=data.index.max(), freq='1h')
#fechas_esperadas =  pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')
fechas_perdidas = fechas_esperadas.difference(data.index)
fechas_perdidas

DatetimeIndex([], dtype='datetime64[ns]', freq='h')

In [32]:
print(f'Tenemos un tota de {len(fechas_perdidas)} fechas perdidas')

Tenemos un tota de 0 fechas perdidas


In [33]:
data = data.reindex(fechas_esperadas)


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52584 entries, 2017-01-01 00:00:00 to 2022-12-31 23:00:00
Freq: h
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   NO (µg/m³N)    47992 non-null  float64
 1   NO2 (µg/m³N)   47992 non-null  float64
 2   SO2 (µg/m³N)   46390 non-null  float64
 3   O3 (µg/m³N)    46626 non-null  float64
 4   TMP (ºC)       51251 non-null  float64
 5   HR (% H.R.)    51251 non-null  float64
 6   NOX (µg/m³N)   47992 non-null  float64
 7   DD (grados)    51251 non-null  float64
 8   PRB (mb)       51251 non-null  float64
 9   RS (W/m³)      51251 non-null  float64
 10  VV (m/s)       51252 non-null  float64
 11  C6H6 (µg/m³N)  14069 non-null  float64
 12  C7H8 (µg/m³N)  14069 non-null  float64
 13  XIL (µg/m³N)   14069 non-null  float64
 14  PM10 (µg/m³N)  49744 non-null  float64
 15  Ruido (dBA)    0 non-null      float64
dtypes: float64(16)
memory usage: 6.8 MB


In [35]:
data.isna().sum()

NO (µg/m³N)       4592
NO2 (µg/m³N)      4592
SO2 (µg/m³N)      6194
O3 (µg/m³N)       5958
TMP (ºC)          1333
HR (% H.R.)       1333
NOX (µg/m³N)      4592
DD (grados)       1333
PRB (mb)          1333
RS (W/m³)         1333
VV (m/s)          1332
C6H6 (µg/m³N)    38515
C7H8 (µg/m³N)    38515
XIL (µg/m³N)     38515
PM10 (µg/m³N)     2840
Ruido (dBA)      52584
dtype: int64

In [36]:
data = data.drop(columns=['Ruido (dBA)'])

In [37]:
data.head()

,NO (µg/m³N),NO2 (µg/m³N),SO2 (µg/m³N),O3 (µg/m³N),TMP (ºC),HR (% H.R.),NOX (µg/m³N),DD (grados),PRB (mb),RS (W/m³),VV (m/s),C6H6 (µg/m³N),C7H8 (µg/m³N),XIL (µg/m³N),PM10 (µg/m³N)
2017-01-01 00:00:00,5.0,30.0,15.0,43.0,10.0,85.0,37.0,299.0,1031.0,2.0,1.0,NaN,NaN,NaN,26.0
2017-01-01 01:00:00,4.0,27.0,12.0,41.0,9.0,84.0,34.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,25.0
2017-01-01 02:00:00,5.0,25.0,14.0,39.0,9.0,86.0,33.0,300.0,1030.0,2.0,1.0,NaN,NaN,NaN,21.0
2017-01-01 03:00:00,3.0,16.0,15.0,43.0,8.0,85.0,21.0,301.0,1029.0,3.0,1.0,NaN,NaN,NaN,18.0
2017-01-01 04:00:00,4.0,15.0,14.0,46.0,8.0,84.0,22.0,302.0,1029.0,2.0,1.0,NaN,NaN,NaN,16.0


In [43]:
duplicates = data.index.duplicated()
print(data.index[duplicates])
print(data.index.is_unique)

DatetimeIndex([], dtype='datetime64[ns]', freq='h')
True


In [39]:
serie_lineal = data.copy()
serie_lineal = serie_lineal.interpolate(method='linear', limit_direction='both')


In [40]:
serie_lineal.isna().sum()

NO (µg/m³N)      0
NO2 (µg/m³N)     0
SO2 (µg/m³N)     0
O3 (µg/m³N)      0
TMP (ºC)         0
HR (% H.R.)      0
NOX (µg/m³N)     0
DD (grados)      0
PRB (mb)         0
RS (W/m³)        0
VV (m/s)         0
C6H6 (µg/m³N)    0
C7H8 (µg/m³N)    0
XIL (µg/m³N)     0
PM10 (µg/m³N)    0
dtype: int64

In [41]:
serie_cuadratica = data.copy()
serie_cuadratica = serie_cuadratica.interpolate(method='linear', limit_direction='both')

In [42]:
serie_cuadratica.isna().sum()

NO (µg/m³N)      0
NO2 (µg/m³N)     0
SO2 (µg/m³N)     0
O3 (µg/m³N)      0
TMP (ºC)         0
HR (% H.R.)      0
NOX (µg/m³N)     0
DD (grados)      0
PRB (mb)         0
RS (W/m³)        0
VV (m/s)         0
C6H6 (µg/m³N)    0
C7H8 (µg/m³N)    0
XIL (µg/m³N)     0
PM10 (µg/m³N)    0
dtype: int64